In [ ]:
# %load load.py
import numpy as np
import pandas as pd
import geopandas as gp
import holoviews as hv
import geoviews as gv
from geoviews import opts

import utils
import importlib
importlib.reload(utils)

gv.extension('bokeh')
opts.defaults(
    opts.Polygons( tools=['hover'], width=600, height=500),
    opts.Points( tools=['hover'], width=600, height=500),
    opts.Overlay(width=600, height=500)
    )

In [208]:
import panel as pn
import param
pn.extension()
from datetime import datetime

Stack

# Keep

#### Footprints

In [169]:
from geoviews import tile_sources as gvts


:WMTS   [Longitude,Latitude]

In [212]:
footprints = gv.Path(gp.read_file('temp/chi_mic.geojson'))
footprints.options(alpha=0)*gvts.EsriReference

:Overlay
   .Path.I :Path   [Longitude,Latitude]
   .WMTS.I :WMTS   [Longitude,Latitude]

#### Roads

In [58]:
roads = gv.Path(gp.read_file('Road Network/RoadNetwork.geojson'))

In [61]:
road_opts = opts.Path(alpha=0.5, color='blue')
roads.options(road_opts)

:Path   [Longitude,Latitude]

# Show

#### Color funcs

In [3]:
def palette_plot(cmap_name, direction=1, provider='colorcet'):

    """
    Searching Holoviews colormap listings by name and provider
    Outputs colorbar
    Specify reverse by direction = -1
    """
    from holoviews.plotting.util import process_cmap

    cmap = process_cmap(cmap=cmap_name, provider=provider)[::direction]
#     return cmbar(cmap).relabel(label=cmap_name)
    return cmap


def cmbar(cmap_hex_list):
    
    """
    Produces colorbar for list of hex colors
    """
    
    import numpy as np
    import holoviews as hv
    
    spacing = np.linspace(0, 1, len(cmap_hex_list))[np.newaxis]
    img_opts = opts.Image(cmap=cmap_hex_list, xaxis='bare', yaxis='bare', xticks=0, yticks=0, toolbar=None,\
                         frame_height=100, frame_width=400)
    return hv.Image(spacing, ydensity=1).opts(img_opts)

In [4]:
def partitioned(basemap, span=50, dl=0.5):
    colrs = []
    for b in basemap:
        addmap = color_fader(adjust_lightness(b, amount=1-dl), b, span) +\
                           color_fader(b, adjust_lightness(b, amount=1+dl),span)
        colrs.extend(addmap)
    return colrs

In [5]:
def color_fader(c1,c2,n): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    import matplotlib as mpl
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return [mpl.colors.to_hex((1-x)*c1 + x*c2) for x in np.linspace(0,1,n)]

In [6]:
def increase(cmap, slc, n_between, n_plus=5, dl=0.5):
    nslices = int(len(cmap)/slc)
    divisions = [cmap[slc*i:slc+slc*i] for i in range(nslices)]
    new_cmap = []
    for d in divisions:
        new_cmap.extend(color_fader(d[0],d[-1],n_between))
    new_cmap = color_fader(adjust_lightness(new_cmap[0], amount=1-dl), new_cmap[0], slc*n_plus) + new_cmap +\
               color_fader(new_cmap[-1], adjust_lightness(new_cmap[-1], amount=1-dl),slc*n_plus)
    return new_cmap

In [7]:
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return mc.to_hex(colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2]))

In [389]:
cmap = palette_plot('Spectral',provider='matplotlib')
(cmbar(cmap) + cmbar(partitioned(cmap[::16],80,0.6))+cmbar(increase(cmap,16, 50))\
+cmbar(cmap[::24])).cols(1)

:Layout
   .Image.I   :Image   [x,y]   (z)
   .Image.II  :Image   [x,y]   (z)
   .Image.III :Image   [x,y]   (z)
   .Image.IV  :Image   [x,y]   (z)

## Outside funcs

In [330]:
make_lookup_dct(next(utils.get_data('weather')).columns).keys()

dict_keys(['Temp', 'Dewpt', 'RH', 'pres', 'RadDir', 'RadDif', 'Longwave', 'Shortwave', 'WindDir', 'WindSpd'])

In [ ]:
def make_lookup_dct(columns):
    lookup_dct = {}
    for i in t.columns:
        name = i.split('_', 1)[0]
        units = i.split('_',1)[1]
        if 'temp' in i:
            name = 'Temp'
        if 'dewpt' in i:
            name = 'Dewpt'
        if 'RadDif' in i:
            name = 'RadDif'
        if 'W' in units:
            units = 'W/m^2'
        if 'ms' in units:
            units = 'm/s'   
        lookup_dct[name] = {}
        lookup_dct[name]['full'] = i
        lookup_dct[name]['units'] = units
    return lookup_dct

In [238]:
def get_cmap_dct(chunks, param, cmap=None, mode='range', darken=0.5, lighten=0.5):
    def slice_cmap(lo,hi, mode=mode, check_total=None):
        get_index = lambda x:int(round(len(cmap)*(x-tot[0])/(tot[1]-tot[0])))
        if mode =='range':
            lo_index, hi_index = get_index(lo), get_index(hi)
            return cmap[lo_index:hi_index]
        elif mode == 'focus':
            ind = get_index(int(round((lo+hi)/2)))
            if ind == len(cmap):
                ind -= 1
            base = cmap[ind]
            return color_fader(adjust_lightness(base, amount=1-darken), base, 50) +\
                   color_fader(base, adjust_lightness(base, amount=1+lighten),50)  
    
    # Get ranges
    cmap_dct = {}
    minmax = []
    for i,j in chunks.items():
        cmap_dct[i] = {}
        seas_max = j[param].max()
        seas_min = j[param].min()
        cmap_dct[i]['values'] = (seas_min, seas_max)
        minmax.extend([seas_min, seas_max])
    cmap_dct['total'] = {}
    cmap_dct['total']['values'] =  min(minmax), max(minmax)
    
    tot = cmap_dct['total']['values']
  
    # Get colors
    tot_colors = []
    for i,j in cmap_dct.items():
        if i == 'total':
            cmap_dct[i]['colors'] = tot_colors
        else:
            add_colors = slice_cmap(*j['values'], mode=mode)
            cmap_dct[i]['colors'] = add_colors
            tot_colors.extend(add_colors)
    
    return cmap_dct

def get_chunk_dct():
    
    if mode == 'daily':
        df = pd.read_csv('temp/avgs/daily_avgs.csv', parse_dates=['Time'])
        unique = set(df['Time'])
        chunk_dct = {i:df[df.Time==i] for i in unique}
    elif mode == 'seasonally':
        seasons = ['winter','spring','summer','fall']
        chunks = [pd.read_csv(f'temp/avgs/{i}.csv') for i in seasons]
        chunks_dct = {s:c.groupby(['Lat','Lon']).mean() for s,c in zip(seasons,chunks)}        

    return chunk_dict

## Class

In [317]:
df = pd.read_csv('temp/avgs/daily_avgs.csv')
unique = set(df['Time'])

In [321]:
pd.read_csv('temp/avgs/daily_avgs.csv', parse_dates=['Time']).Time[0]

Timestamp('2015-01-01 00:00:00')

In [238]:
def get_cmap_dct(chunks, params, cmap=None, mode='range', darken=0.5, lighten=0.5):
    def slice_cmap(lo,hi, mode=mode, check_total=None):
        get_index = lambda x:int(round(len(cmap)*(x-tot[0])/(tot[1]-tot[0])))
        if mode =='range':
            lo_index, hi_index = get_index(lo), get_index(hi)
            return cmap[lo_index:hi_index]
        elif mode == 'focus':
            ind = get_index(int(round((lo+hi)/2)))
            if ind == len(cmap):
                ind -= 1
            base = cmap[ind]
            return color_fader(adjust_lightness(base, amount=1-darken), base, 50) +\
                   color_fader(base, adjust_lightness(base, amount=1+lighten),50)  
    
    # Get ranges
    cmap_dct = {}
    minmax = []
    for i,j in chunks.items():
        cmap_dct[i] = {}
        for p in params:
            cmap_dct[i][p] = {}
            seas_max = j[param].max()
            seas_min = j[param].min()
            cmap_dct[i][p]['values'] = (seas_min, seas_max)
            minmax.extend([seas_min, seas_max])
        cmap_dct[i][p]['total'] = {}
        cmap_dct[i][p]['total']['values'] =  min(minmax), max(minmax)
    
    tot = cmap_dct['total']['values']
  
    # Get colors
    tot_colors = []
    for i,j in cmap_dct.items():
        if i == 'total':
            cmap_dct[i]['colors'] = tot_colors
        else:
            add_colors = slice_cmap(*j['values'], mode=mode)
            cmap_dct[i]['colors'] = add_colors
            tot_colors.extend(add_colors)
    
    return cmap_dct

In [336]:
class Example(param.Parameterized):
    """An example Parameterized class"""
    select_string           = param.ListSelector(default=[3, 5], objects=[1, 3, 5, 7, 9], precedence=0.5)


In [341]:
a = Example(select_string=[1,3])

In [342]:
a.select_string

[1, 3]

In [ ]:
    
    def grid_data(time, chunk, variables, ranges):
        var_plots = []

        ### Set Options ###
        cmap = ranges[time]['colors']
        gen_opts = opts.Points(frame_width=200, frame_height=200, alpha=1, padding=(0,0,0),
                               colorbar=True, toolbar='above', xaxis=None, yaxis=None, 
                               size=13, cmap=cmap, marker='s', axiswise=True)
        for v in variables:

            ### Normalize color bar
            chunk = chunk.rename(columns={v:'tmp'}) # switch to temporary val to get 
            title = season + ': ' + v.split('_')[0]
            var_plot = gv.Points(chunk, ['Lon', 'Lat'], vdims='tmp').opts(title=title)#season)
            var_plot = var_plot.redim(tmp=dict(range=self.color_ranges[season]['values']))
            var_plot.opts(gen_opts).opts(clabel='W/m2', color_index='tmp') 
            var_plot.redim(tmp=v)              
            chunk = chunk.rename(columns={'tmp':v})
            output = var_plot*footprints.opts(alpha=0.3, color='black')
    #         display(output)
            var_plots.append(output)

        return hv.Layout(var_plots).opts( toolbar=None, shared_axes=False)

In [327]:
import warnings
warnings.filterwarnings('ignore')

class Weather(param.Parameterized):
    
    import colorcet as cc
    
    # user inputs mode
    mode = param.String(default='seasonally') # seasonally or daily
    variables = ['Temp', 'Dewpt', 'RH', 'pres', 'RadDir', 'RadDif', 
                     'Longwave', 'Shortwave', 'WindDir', 'WindSpd']
    variables = param.ListSelector(default=['Temp'], objects=variables)
    initialize=False
    
    # select number of curves and max pks/curve for sample data
    if mode == 'daily':
        dt = datetime.fromisoformat
        time = param.Date(default=dt('2015-01-01'), bounds=(dt('2015-01-01'), dt('2015-12-31')))
    elif mode == 'seasonally':
        time = param.Selector(objects=['winter','spring','summer','fall'])
    plots = None

    # handle weather data
    lookup_dct = make_lookup_dct(chunks.values()[0]) # assumes all chunks have same columns
    cmap = cc.fire[120::24]
 
    # initializes mode
    def initialize(self):
        chunks =  get_files(self.mode)
        self.chunk_dct = get_chunk_dct(self.mode)
        self.timeunit = sorted(set([i[0] for i in chunk_dict.keys()]))
        self.color_ranges = get_cmap_dct(chunks, 'temp_K', self.cmap, darken=0.5, lighten=2, mode='focus')
        
    # Final output
    @param.depends('time')
    def view(self):
        if not self.initialized:
            self.initialize()
        time_widg =  pn.Param( self.param.time, widgets={'time':{'type':pn.widgets.DateSlider}} )
        return pn.Column(time_widg, self.plot)
    
    def plot(self):
        
        if self.mode == 'seasonally':
        
        elif self.mode == 'daily':
        
            
        try:
            dfs = self.chunk_dict[self.time]
            self.plots = grid_data(dfs, variables)
        except KeyError:
            return 'No available data for this data and time.'

            return self.plots
    

    def animate(self):
        
        import time

        for t in self.timeunit:
            self.time = t
            time.sleep(0.05)
 

w = Weather()
w.view()

IndentationError: unexpected indent (<ipython-input-327-f1dc76d889fa>, line 74)

In [ ]:
%%capture
w.animate()